# Extract profiles for functional groups of the IUCN Global Ecosystem typology

Scripts by José R. Ferrer-Paris

The scripts described in this document are used to:

- Read data from database
- Write static markdown document with summary of all profiles

## Set-up
Load all the libraries we will need in this script:

In [3]:
import os
from pathlib import Path
import re
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs
import shutil
import yaml

In [5]:

# Pyprojroot for easier handling of working directory
import pyprojroot
repodir = pyprojroot.find_root(pyprojroot.has_dir(".git"))


Read configuration parameters for the connection to the current version of the database:

In [9]:
filename = Path(repodir / 'secrets' / 'database.ini')
section = 'altaws'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

## Connect to database

In [10]:
conn = psycopg2.connect(**params)
cur = conn.cursor(cursor_factory=DictCursor)


In [15]:
outfile =  repodir  / 'output' / 'mapinfo.yaml'
qry="""
    SELECT map_code, map_version, code, map_source,contributors,status 
    FROM map_metadata WHERE status IN ('valid','replaced','superceded','superceeded') 
    ORDER BY map_version DESC;
    """
cur.execute(qry)
mapinfo = cur.fetchall()
records=dict()
record=dict()
record['code']='MISSING'
record['version']='MISSING'
record['description']='Map information is missing'
record['contributors']='MISSING'
records['MISSING']=record
for item in mapinfo:
    mapcode="%s_%s" % (item['map_code'],item['map_version'])
    record=dict()
    record['code']=item['map_code']
    record['efg']=item['code']
    record['version']=item['map_version']
    record['description']=item['map_source']
    record['contributors']=", ".join(item['contributors'])
    if item['status']=='valid':
        record['status']='valid'
    else:
        record['status']='replaced'
    records[mapcode]=record

with open(outfile, 'w') as file:
    yaml.dump(records, file)


In [13]:
outfile =  repodir / 'output' / 'maprefs.yaml'
qry="""
    SELECT map_code,map_version,ARRAY_AGG(ref_code) as refs, ARRAY_AGG(dataset) as datasets
    FROM map_references 
    GROUP BY map_code, map_version;
    """
cur.execute(qry)
refs = cur.fetchall()

records=dict()
records['MISSING']={'references':'Missing references'}
for item in refs:
    mapcode="%s_%s" % (item['map_code'],item['map_version'])
    records[mapcode]={'references':item['refs'],
                      'datasets':item['datasets']}

with open(outfile, 'w') as file:
    yaml.dump(records, file)

In [14]:
outfile =  repodir / 'output' / 'efgrefs.yaml'
qry="""
    SELECT code,ARRAY_AGG(ref_code) as refs 
    FROM efg_references 
    GROUP BY CODE
    """
cur.execute(qry)
refs = cur.fetchall()

records=dict()
records['MISSING']=('Missing references',)
for item in refs:

    records[item['code']]=item['refs']

with open(outfile, 'w') as file:
    yaml.dump(records, file)

## Update EFG content (short descriptions)

In [16]:
qry="""
SELECT distinct code
FROM functional_groups f
LEFT JOIN biomes b
    ON b.biome_code=f.biome_code
ORDER BY code
;"""
cur.execute(qry)
efgs = cur.fetchall()

In [17]:
efgs[0]

['F1.1']

In [18]:
sections = ("Ecological Traits","Key Ecological Drivers","Distribution")
version = 'v2.1'
records=list()
for efg in efgs:
    qry="""
    SELECT code, f.biome_code as biome_code, f.name as name, f.shortname as shortname, b.name as biome_name,
    realms, f.update as original_date, shortdesc, keyfeatures, distdesc 
    FROM functional_groups f
    LEFT JOIN biomes b
        ON b.biome_code=f.biome_code
    WHERE code = %s
    ;"""
    cur.execute(qry,(efg[0],))

    efginfo = cur.fetchone()
    record=dict()
    
    record['name']=efginfo['name']
    record['biome']=efginfo['biome_name']
    record['realm']=efginfo['realms']
    record['code']=efginfo['code']
    record['biomecode']=efginfo['biome_code']
    record['shortdesc']=efginfo['shortdesc']
    record['keyfeatures']=efginfo['keyfeatures']
    record['distdesc']=efginfo['distdesc']
    
    record['contributors']=list()
    
    for section in sections:
        qry ="""
        SELECT description,contributors,version,update 
        FROM efg_%s 
        WHERE code = %s AND language = 'en' AND version=%s
        ORDER BY update DESC;
        """
        cur.execute(qry,(AsIs(section.lower().replace(' ','_')),efginfo['code'],version))
        sectinfo = cur.fetchone()
        record[section]=sectinfo['description']
        for author in sectinfo['contributors']:
            if author not in record['contributors']:
                record['contributors'].append(author)
        record['version']="%s (%s)" % (sectinfo['version'],sectinfo['update'].date())

    reflist1=list()
    reflist2=list()

    
    qry="""
    SELECT map_code, map_version 
    FROM map_metadata WHERE code = %s AND status='valid' AND map_type='Indicative Map' 
    ORDER BY map_version DESC;
    """
    cur.execute(qry,(efg[0],))
    mapinfo = cur.fetchone()
    if mapinfo is not None:
        record['mapcode']="%s_%s" % (mapinfo['map_code'],mapinfo['map_version'])
    else:
        record['mapcode']="MISSING" 


    record['realmstr']=", ".join(record['realm'])
    record['contributorstr']=", ".join(record['contributors'])
    
    qry="""
    SELECT ref_cite 
    FROM efg_references as e 
    LEFT JOIN ref_list as l ON e.ref_code=l.ref_code 
    WHERE code = %s
    ORDER BY ref_cite;
    """
    cur.execute(qry,(efg[0],))
    refs = cur.fetchall()
    
    for ref in refs:
        if ref[0] is not None:
            reflist1.append("* "+ref[0])
    
    
    #record['maprefstr']='{% for ref in map.contributors %}\n* {{ref}}\n{% endfor %}'
    
    #record['mainrefstr']="\n".join(reflist1)
    
    records.append(record)

In [22]:
records[0]

{'name': 'F1.1 Permanent upland streams',
 'biome': 'F1. Rivers and streams biome',
 'realm': ['Freshwater'],
 'code': 'F1.1',
 'biomecode': 'F1',
 'shortdesc': 'These small rivers or streams in mountainous or hilly areas are characterised by steep gradients and fast flow. They flow all year, increasing in wet periods, in humid tropical and temperate zones. Stones are common along their rapids and pools, turning over and oxygenating the water. Dependent organisms are specialised for these high flow-velocity environments, with resources for food webs derived mainly from the stream and inputs from adjacent and upstream vegetation.',
 'keyfeatures': 'High-medium velocity, low-medium volume perennial flows with abundant benthic filter feeders, algal biofilms & small fish',
 'distdesc': 'Global uplands with wet climates',
 'contributors': ['RT Kingsford',
  'R Mac Nally',
  'PS Giller',
  'MC Rains',
  'M Kelly-Quinn',
  'AH Arthington',
  'DA Keith'],
 'Ecological Traits': 'These 1st-3rd o

In [20]:
template = """
# {name}

In the IUCN Global Ecosystem Typology this ecosystem functional group is identified as *{name}*.
It belongs to the {biome}, and is part of the {realmstr} realm.

{shortdesc} {distdesc}

The profile for this ecosystem functional group was written by {contributorstr}. 
The version {version} of this profile is available at
<https://global-ecosystems.org/explore/groups/{code}>

## Key Features

{keyfeatures}

## Ecological traits

{Ecological Traits}

## Key Ecological Drivers

{Key Ecological Drivers}

## Distribution

{Distribution}

"""

In [24]:
outfile =  repodir / 'output' / 'Ecosystem-profiles-short-descriptions.md'
for record in records[1:25]:
    outtext=template.format(**record)
    with open(outfile,'a') as f:
        f.write(outtext)

## Close database connection

In [25]:

cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


And then we can do something like directly with pandoc:

This could work with `quarto` if `tex` (or at least `tinytex`) was installed: